<div style="display: flex; align-items: center;">
    <img src="https://github.com/nagelt/Teaching_Scripts/raw/9d9e29ecca4b04eaf7397938eacbf116d37ddc93/Images/TUBAF_Logo_blau.png" width="500" height="auto" height="auto" style="margin-right: 100px;" />
    <div>
        <p><strong>Prof. Dr. Thomas Nagel</strong></p>
        <p>Chair of Soil Mechanics and Foundation Engineering<br>Geotechnical Institute<br>Technische Universität Bergakademie Freiberg.</p>
        <p><a href="https://tu-freiberg.de/en/soilmechanics">https://tu-freiberg.de/en/soilmechanics</a></p>
    </div>
</div>

# Passive Earth Pressure Coulomb

![Erddruck_passiv.png](https://raw.githubusercontent.com/nagelt/Teaching_Scripts/master/Images/Erddruck_passiv.png)

This is an educational example only. The assumption of planar slip surfaces leads to an overestimation of passive earth pressures. Curved slip surfaces should be used instead.

## Geometry 
Let the retaining wall have a vertical height $H$ and the origin be placed at the bottom of the slip surface which has length $L$. Then, the problem is characterized by the following points in the $x-z$ plane:

* Origin: $O = [0,0]$
* Top left: $A = [-H \tan \alpha, H]$
* Top right: $B = [L\cos \vartheta, L \sin \vartheta]$

The length of the slip surface can be found via the following trigonometric relation:

\begin{align}
    L \sin \vartheta &= (L \cos \vartheta + H \tan\alpha) \tan \beta
\end{align}

Thus we arrive at

\begin{align}
    L &= \frac{H(1 + \tan \alpha \tan \beta)}{\sin \vartheta - \cos \vartheta \tan \beta}
\end{align}

The area of the triangle (i.e. the volume of the slip body per meter) can also be calculated. We use Gauß' equation for that:

$$
    A = \frac{1}{2} \left| (x_2 y_3 - x_3 y_2) - (x_1 y_3 - x_3 y_1) + (x_1 y_2 - x_2 y_1) \right|
$$

Let Python do the math for us ...

In [1]:
#HIDDEN
import numpy as np #numerical methods
#import sympy as sp #symbolic operations
import matplotlib.pyplot as plt #plotting
import sympy as sp
sp.init_printing(use_latex='mathjax') #makes sympy output look nice

#Some plot settings
import plot_functions.plot_settings
%run plot_functions/earth_pressure_plots.ipynb

We first define our geometrical properties:

In [2]:
H,alpha,beta,theta = sp.symbols('H alpha beta vartheta')

In [3]:
L = H*(1+sp.tan(alpha)*sp.tan(beta))/(sp.sin(theta) - sp.cos(theta) * sp.tan(beta))
L

       H⋅(tan(α)⋅tan(β) + 1)        
────────────────────────────────────
sin(vartheta) - cos(vartheta)⋅tan(β)

We now define the function that calculates the area of the triangle:

In [4]:
def triangle_area_Gauss(A,B,C):
    term1 = (B[0]*C[1] - C[0] * B[1])
    term2 = (A[0]*C[1] - C[0] * A[1])
    term3 = (A[0]*B[1] - B[0] * A[1])
    return -(term1 - term2 + term3)/2

The points we defined above can now be inserted into this equation:

In [5]:
PO = [0,0]
PA = [- H * sp.tan(alpha), H]
PB = [L * sp.cos(theta), L*sp.sin(theta)]
A = triangle_area_Gauss(PO,PA,PB).simplify()
A

 2                                                           
H ⋅(sin(vartheta)⋅tan(α) + cos(vartheta))⋅(tan(α)⋅tan(β) + 1)
─────────────────────────────────────────────────────────────
           2⋅(sin(vartheta) - cos(vartheta)⋅tan(β))          

Thus, the area of our triangle is given as:

$$
    A = \frac{H^2 (\sin \vartheta \tan \alpha + \cos \vartheta) (\tan \alpha \tan \beta + 1)}{2(\sin \vartheta - \cos \vartheta \tan \beta)}
$$

Now we can write the force balance. For that purpose, we introduce the material's specific weight $\gamma$, friction angle $\varphi$ and the wall friction angle $\delta_\text{a}$ as well as the cohesion $c$.

In [6]:
c, delta, phi, gamma = sp.symbols('c delta_p varphi gamma')

We can now calculate the downward force due to gravity as well as the cohesive force along the slip surface. The passive earth pressure force $E_\text{p}$ and the reaction force $Q$ in the slip line are introduced as unknowns:

In [7]:
Ep, Q, G, C = sp.symbols('E_p Q G C')

We now write the vertical and horizontal equilibrium conditions:

\begin{align}
    \text{horizontal:}\quad  0 &= -C \cos\vartheta - Q \sin (\vartheta + \varphi) + E_\text{p} \cos (\alpha - \delta_\text{p})
    \\
    \text{vertikal:}\quad  0 &= -C \sin\vartheta + Q \cos (\vartheta + \varphi) + E_\text{p} \sin (\alpha - \delta_\text{p}) - G
\end{align}

In [8]:
Fh = sp.Eq(-C * sp.cos(theta) - Q * sp.sin(theta + phi) + Ep * sp.cos(alpha - delta),0)
Fv = sp.Eq(-C * sp.sin(theta) + Q * sp.cos(theta + phi) + Ep * sp.sin(alpha - delta) - G,0)

In [9]:
Fh

-C⋅cos(vartheta) + Eₚ⋅cos(α - δₚ) - Q⋅sin(varphi + vartheta) = 0

In [10]:
Fv

-C⋅sin(vartheta) + Eₚ⋅sin(α - δₚ) - G + Q⋅cos(varphi + vartheta) = 0

Let's find a symbolic solution.

In [11]:
result = sp.solve([Fh,Fv],(Ep,Q))

In [12]:
result

⎧                     C⋅sin(vartheta)⋅sin(varphi + vartheta)                  
⎨Eₚ: ─────────────────────────────────────────────────────────────────────── +
⎩    sin(α - δₚ)⋅sin(varphi + vartheta) + cos(α - δₚ)⋅cos(varphi + vartheta)  

                  C⋅cos(vartheta)⋅cos(varphi + vartheta)                      
 ─────────────────────────────────────────────────────────────────────── + ───
 sin(α - δₚ)⋅sin(varphi + vartheta) + cos(α - δₚ)⋅cos(varphi + vartheta)   sin

                     G⋅sin(varphi + vartheta)                                 
────────────────────────────────────────────────────────────────────, Q: ─────
(α - δₚ)⋅sin(varphi + vartheta) + cos(α - δₚ)⋅cos(varphi + vartheta)     sin(α

                 C⋅sin(vartheta)⋅cos(α - δₚ)                                  
────────────────────────────────────────────────────────────────── - ─────────
 - δₚ)⋅sin(varphi + vartheta) + cos(α - δₚ)⋅cos(varphi + vartheta)   sin(α - δ

             C⋅sin(α - δₚ)⋅cos(vartheta)        

In [13]:
result.get(Ep).simplify()

C⋅cos(varphi) + G⋅sin(varphi + vartheta)
────────────────────────────────────────
    cos(-α + δₚ + varphi + vartheta)    

In [14]:
result.get(Q).simplify()

C⋅sin(-α + δₚ + vartheta) + G⋅cos(α - δₚ)
─────────────────────────────────────────
     cos(-α + δₚ + varphi + vartheta)    

Gravitational and cohesive forces can be determined based on material properties and geometry:

In [15]:
result.get(Ep).subs(C,0).subs(G,A*gamma).simplify()

 2                                                                            
H ⋅γ⋅(sin(vartheta)⋅tan(α) + cos(vartheta))⋅(tan(α)⋅tan(β) + 1)⋅sin(varphi + v
──────────────────────────────────────────────────────────────────────────────
      2⋅(sin(vartheta) - cos(vartheta)⋅tan(β))⋅cos(-α + δₚ + varphi + vartheta

        
artheta)
────────
)       

Now we would have to find the minimum with respect to $\vartheta$ in order to identify the passive earth pressure and the critical slip angle $\vartheta_\text{p}$. The equations get very complex and we decide to go for a numerical solution.

In [16]:
plot_interactive_earth_pressure_passive()

interactive(children=(IntSlider(value=10, description='$c$ / kN m$^{-2}$:', max=50, step=5), IntSlider(value=3…

## Tasks

* Compare the numbers you get to results obtained from the analytical expressions you find in the current norm.
* Investigate the influence of the different parameters.

![Erddruck_passiv.png](https://raw.githubusercontent.com/nagelt/Teaching_Scripts/master/Images/Erddruck_passiv.png)